# Only Parallel Data Model

This is the Python Notebook concerning the final version containing the coding for the Only-Parallel (Supervised) version of the program. It is to be completed with the correct format of the Contextual and Style Classifiers in their corresponding parts.

Date of upload: Friday 31th January

Actual Version: 1.1 (Friday 7th February)

Precedent Versions : 1.0



## import(s)

In [1]:
from scripts.data_builders.prepare_dataset import prepare_dataset,string2code,code2string

import math
import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ",device)

device =  cpu


## Data Pre-processing

In [4]:
train_data, dict_words = prepare_dataset(device,ratio=0.5,shuffle_ctx=False) #check with shift+tab to look at the data structure
batch_size = 64
dict_token = {b:a for a,b in dict_words.items()}

Loading ...
- Shakespeare dataset length :  20316
- Corrupted samples (ignored) :  763


## Parameters

In [ ]:
nb_heads = 4
d_feedforward = 1024
batch_size = 64

d_embedding = 784

## Positional Encoding

In [3]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## Context Classifier (To be filled)

## Style classifier (To be filled)

## Model (To be adapted once we have the Context and Style Class For the Embedding)

In [ ]:
class ParallelModel(torch.nn.Module):
    def __init__(self,dict_size, d_embedding,nb_heads, d_feedforward):
        super().__init__()
        
        self.embed_layer = torch.nn.Embedding(dict_size+1, d_embedding, padding_idx=dict_size)
        self.positional_layer = PositionalEncoding(d_embedding)
        self.sentence_encoder = torch.nn.TransformerEncoderLayer(d_model = d_embedding, nhead = nb_heads, 
                                                    dim_feedforward = d_feedforward)
        self.context_encoder = torch.nn.TransformerEncoderLayer(d_model = d_embedding, nhead = nb_heads, 
                                                    dim_feedforward = d_feedforward)
        self.sentence_decoder = torch.nn.TransformerDecoderLayer(d_model = d_embedding + 1, nhead = nb_heads, 
                                                    dim_feedforward = d_feedforward)
        self.label_embedding = torch.nn.Embedding(2,)
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(),
            torch.nn.LogSoftmax(dim=2))
    
    
    def _generate_square_subsequent_mask(sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    def forward(self,x,ctx_x,y,label_y):
        device = x.device
        mask_x = self._generate_square_subsequent_mask(x.shape[1]).to(device)
        mask_ctx = self._generate_square_subsequent_mask(ctx_x.shape[1]).to(device)
        mask_y = self._generate_square_subsequent_mask(y.shape[1]).to(device)
        
        # Embedding
        x = self.embed_layer(x).transpose(0,1) # token x batch x embedding
        ctx_x = self.embed_layer(ctx_layer).transpose(0,1)
        
        # Positional Encoding
        x = self.positional_layer(x)
        ctx_x = self.positional_layer(x)
        
        # Encoders
        x_enc = self.sentence_encoder(x,mask_x)
        ctx_enc = self.context_encoder(x,mask_ctx)
        
        # Linear and Style Mixing
        x_and_ctx = torch.cat((x_enc,ctx_enc),dim = 0)
        label = label.reshape((1,label.shape[0])).expand((x_and_ctx.shape[0],x_and_ctx.shape[1]))
        x_lab = x_and_ctx + self.label_embedding(label)
        
        # Decoder
        y = embed_layer(y)
        y_pos = self.positional_layer(y.transpose(0,1))
        y_pred = sentence_decoder(y_pos,x_lab,mask_y)
        
        return(y_pred.transpose(0,1),y)

## Training  (To be adapted once we have the Context and Style Class For the Embedding)

In [ ]:
# Definition of the model(s)

model = ParallelModel(dict_size, d_embedding, nb_heads, d_feedforward)

In [ ]:
# Information concerning the Training optimizer

from torch.optim import Adam

params = list(sentence_encoder.parameters()) + list(context_encoder.parameters()) 
        + list(linear_context.parameters) + list(sentence_decoder.parameters())

l_r = 5e-4
optimizer=Adam(params,lr=l_r)
l1=1 #
l2=1
l3=1

In [ ]:
# Losses 
loss_seq2seq = torch.nn.SmoothL1Loss() #Contextual Seq2Seq Loss
loss_coherence = torch.nn.CrossEntropyLoss() #Contextual Coherence Loss
loss_style = torch.nn.BCELoss() #Style Classification Loss

In [ ]:
nb_epoch = 100
for epoch in range(nb_epoch):
    for x, y, ctx_x,ctx_y , len_x,len_y , len_ctx_x,len_ctx_y, label,label_ctx in train_loader:
        y_pred, y = model.forward(x,ctx_x,y,label) #Output still embedded 
        
        # Seq2Seq Loss with Token
        
        loss_seq = l1*loss_seq2seq(y_pred,y)
        
        # Argmax
        y_pred = torch.argmax(softmax_layer(LogSoftmax),dim = 2)
        
        # Context + Style class
        
        